In [1]:
import polars as pl
from polars import selectors as cs

In [2]:
df = pl.read_parquet("data/preprocessed_data.parquet")
print(df.shape)
df.head()

(51336, 49)


PROSPECTID,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,tot_enq,CC_enq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,MARITALSTATUS,EDUCATION,AGE,GENDER,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_of_active_TLs_ever,pct_opened_TLs_L6m_of_L12m,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
i64,f64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,str,i64,i64,f64,f64,i64,i64,f64,f64,i64,i64,str,str,i64,str
1,0.0,0,0.0,0.0,0,0,1,0,4,1,4,0,72,18,549,0,0,11,0,0,0,0,0,29,6,0,0,0,566,0,"""Married""","""12TH""",48,"""M""",51000,114,0.2,0.0,0,1,0.0,0.0,1,0,"""PL""","""PL""",696,"""P2"""
2,0.0,0,1.0,0.0,0,0,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,1,0,0,0,209,0,"""Single""","""GRADUATE""",23,"""F""",19000,50,1.0,0.0,0,0,0.0,0.0,0,0,"""ConsumerLoan""","""ConsumerLoan""",685,"""P2"""
3,0.0,0,0.25,0.0,1,0,1,0,0,2,6,0,47,2,302,8,0,10,0,0,0,0,0,25,4,0,0,0,587,0,"""Married""","""SSC""",40,"""M""",18,191,1.0,0.5,0,0,0.0,0.0,1,0,"""ConsumerLoan""","""others""",693,"""P2"""
4,0.0,0,1.0,0.0,1,0,0,0,0,0,1,1,5,5,null,0,0,5,0,0,0,0,0,0,null,null,null,null,null,null,"""Married""","""SSC""",34,"""M""",10000,246,1.0,1.0,0,0,0.0,0.0,0,0,"""others""","""others""",673,"""P2"""
5,0.0,0,0.0,0.0,0,0,0,0,0,3,0,2,131,32,583,0,0,16,0,0,0,0,0,0,1,0,0,0,3951,0,"""Married""","""POST-GRADUATE""",48,"""M""",15000,75,0.333,0.0,0,0,0.0,0.0,0,0,"""AL""","""AL""",753,"""P1"""


In [3]:
# segregate cols of dataset
target_col = "Approved_Flag"
cat_cols = df.select(cs.string().exclude(target_col)).columns
num_cols = df.select(cs.numeric().exclude(target_col, "PROSPECTID")).columns

print(f"{target_col = }", end="\n\n")
print(f"{len(cat_cols)} {cat_cols = }", end="\n\n")
print(f"{len(num_cols)} {num_cols = }", end="\n\n")

target_col = 'Approved_Flag'

5 cat_cols = ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

42 num_cols = ['pct_tl_closed_L6M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment', 'num_deliq_6_12mts', 'num_times_60p_dpd', 'num_std_12mts', 'num_sub', 'num_sub_12mts', 'num_dbt', 'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'tot_enq', 'CC_enq', 'CC_enq_L12m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'AGE', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'pct_of_active_TLs_ever', 'pct_opened_TLs_L6m_of_L12m', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever', 'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'Credit_Score']



In [4]:
for col in cat_cols:
    print(f"{col:>15} - {df.get_column(col).unique().to_list()}")

  MARITALSTATUS - ['Single', 'Married']
      EDUCATION - ['POST-GRADUATE', 'OTHERS', 'PROFESSIONAL', '12TH', 'UNDER GRADUATE', 'GRADUATE', 'SSC']
         GENDER - ['F', 'M']
 last_prod_enq2 - ['others', 'PL', 'CC', 'HL', 'ConsumerLoan', 'AL']
first_prod_enq2 - ['ConsumerLoan', 'PL', 'AL', 'others', 'HL', 'CC']


In [5]:
ORD_COLS = [
    "EDUCATION",
]

OHE_COLS = [
    "MARITALSTATUS",
    "GENDER",
    "last_prod_enq2",
    "first_prod_enq2",
]

In [6]:
EDUCATION_ORD = [
    "SSC",
    "12TH",
    "OTHERS",
    "GRADUATE",
    "UNDER GRADUATE",
    "PROFESSIONAL",
    "POST-GRADUATE",
]

## The Pipeline


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
from sklearn.svm import SVC

## Split Data


In [8]:
final_df = df.drop_nulls()

In [9]:
X = final_df.drop(target_col).to_pandas()
y = final_df.get_column(target_col).to_pandas()

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
)

In [10]:
X_train.shape, y_train.shape

((31574, 48), (31574,))

## Model Building


### Preprocessing `ColumnTransformer`


In [11]:
# This is preprocessing pipeline (valid for every models)
column_trf = ColumnTransformer(
    [
        ("scaler", StandardScaler(), num_cols),
        (
            "ohe",
            OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False),
            OHE_COLS,
        ),
        ("ord", OrdinalEncoder(categories=[EDUCATION_ORD]), ORD_COLS),
    ],
    remainder="passthrough",
)

### Important functions


In [12]:
def perform_cv_with_pipe(pipe: Pipeline, X, y, *, cv: int = 8) -> float:
    cv_scores = cross_val_score(pipe, X, y, scoring="accuracy", cv=cv, verbose=3)
    return float(cv_scores.mean())

In [13]:
def cls_report_with_pipe(pipe: Pipeline, X_test, y_test):
    y_pred = pipe.predict(X_test)
    return classification_report(y_test, y_pred)

### `RandomForestClassifier`


In [14]:
rf_pipe = Pipeline(
    [
        ("column_trf", column_trf),
        ("model", RandomForestClassifier()),
    ]
)

In [15]:
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('column_trf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  ['pct_tl_closed_L6M',
                                                   'Tot_TL_closed_L12M',
                                                   'pct_tl_open_L12M',
                                                   'pct_tl_closed_L12M',
                                                   'Tot_Missed_Pmnt', 'CC_TL',
                                                   'Gold_TL', 'Home_TL',
                                                   'PL_TL', 'Secured_TL',
                                                   'Unsecured_TL', 'Other_TL',
                                                   'Age_Oldest_TL',
                                                   'Age_Newest_TL',
                                                   'time_since_recent_payment',
                                                   'n...
                                                   'time_since_recent_enq',
                                                   'enq_L3m', ...]),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['MARITALSTATUS', 'GENDER',
                                                   'last_prod_enq2',
                                                   'first_prod_enq2']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['SSC',
                                                                              '12TH',
                                                                              'OTHERS',
                                                                              'GRADUATE',
                                                                              'UNDER '
                                                                              'GRADUATE',
                                                                              'PROFESSIONAL',
                                                                              'POST-GRADUATE']]),
                                                  ['EDUCATION'])])),
                ('model', RandomForestClassifier())])

In [16]:
y_pred = rf_pipe.predict(X_test)

In [19]:
y_pred = pl.Series("y_pred", y_pred)
y_pred.head()

y_pred
str
"""P2"""
"""P2"""
"""P2"""
"""P4"""
"""P2"""
"""P3"""
"""P2"""
"""P1"""
"""P2"""


In [22]:
y_pred.value_counts()

y_pred,count
str,u32
"""P3""",1518
"""P1""",1295
"""P4""",1309
"""P2""",6403


In [23]:
y_test.value_counts()

Approved_Flag
P2    6397
P3    1595
P4    1309
P1    1224
Name: count, dtype: int64

In [27]:
perform_cv_with_pipe(rf_pipe, X, y)

[CV] END ................................ score: (test=0.992) total time=   4.7s
[CV] END ................................ score: (test=0.991) total time=   4.7s
[CV] END ................................ score: (test=0.991) total time=   5.0s
[CV] END ................................ score: (test=0.990) total time=   4.8s
[CV] END ................................ score: (test=0.999) total time=   4.8s


0.9925177977987243

In [24]:
print(cls_report_with_pipe(rf_pipe, X_test, y_test))

              precision    recall  f1-score   support

          P1       0.94      1.00      0.97      1224
          P2       1.00      1.00      1.00      6397
          P3       1.00      0.95      0.98      1595
          P4       1.00      1.00      1.00      1309

    accuracy                           0.99     10525
   macro avg       0.99      0.99      0.99     10525
weighted avg       0.99      0.99      0.99     10525



### `LogisticRegression`


In [43]:
logit_pipe = Pipeline(
    [
        ("preprocessing", column_trf),
        ("logistic", LogisticRegression(max_iter=1000)),
    ]
)
logit_pipe.fit(X_train, y_train)

/Users/iarv/Developer/notebooks/credit-modeling/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  ['pct_tl_closed_L6M',
                                                   'Tot_TL_closed_L12M',
                                                   'pct_tl_open_L12M',
                                                   'pct_tl_closed_L12M',
                                                   'Tot_Missed_Pmnt', 'CC_TL',
                                                   'Gold_TL', 'Home_TL',
                                                   'PL_TL', 'Secured_TL',
                                                   'Unsecured_TL', 'Other_TL',
                                                   'Age_Oldest_TL',
                                                   'Age_Newest_TL',
                                                   'time_since_recent_payment'...
                                                   'time_since_recent_enq',
                                                   'enq_L3m', ...]),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['MARITALSTATUS', 'GENDER',
                                                   'last_prod_enq2',
                                                   'first_prod_enq2']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['SSC',
                                                                              '12TH',
                                                                              'OTHERS',
                                                                              'GRADUATE',
                                                                              'UNDER '
                                                                              'GRADUATE',
                                                                              'PROFESSIONAL',
                                                                              'POST-GRADUATE']]),
                                                  ['EDUCATION'])])),
                ('logistic', LogisticRegression(max_iter=1000))])

In [44]:
perform_cv_with_pipe(logit_pipe, X, y)

# NOTE:
# 1. Each iteration take ``approx. 15 secs``
# 2. Model score is lesser than RandomForest

/Users/iarv/Developer/notebooks/credit-modeling/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................................ score: (test=0.879) total time=  15.5s


/Users/iarv/Developer/notebooks/credit-modeling/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................................ score: (test=0.842) total time=  14.8s


/Users/iarv/Developer/notebooks/credit-modeling/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................................ score: (test=0.823) total time=  14.7s


/Users/iarv/Developer/notebooks/credit-modeling/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................................ score: (test=0.846) total time=  18.2s
[CV] END ................................ score: (test=0.854) total time=  18.2s


/Users/iarv/Developer/notebooks/credit-modeling/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8486426076748131

In [46]:
print(cls_report_with_pipe(logit_pipe, X_test, y_test))

              precision    recall  f1-score   support

          P1       0.82      0.78      0.80      1224
          P2       0.86      0.96      0.91      6397
          P3       0.65      0.42      0.51      1595
          P4       0.83      0.78      0.80      1309

    accuracy                           0.83     10525
   macro avg       0.79      0.73      0.76     10525
weighted avg       0.82      0.83      0.82     10525



### `svm.SVC`


In [47]:
svm_pipe = Pipeline(
    [
        ("preprocessing", column_trf),
        ("svm", SVC()),
    ]
)
svm_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  ['pct_tl_closed_L6M',
                                                   'Tot_TL_closed_L12M',
                                                   'pct_tl_open_L12M',
                                                   'pct_tl_closed_L12M',
                                                   'Tot_Missed_Pmnt', 'CC_TL',
                                                   'Gold_TL', 'Home_TL',
                                                   'PL_TL', 'Secured_TL',
                                                   'Unsecured_TL', 'Other_TL',
                                                   'Age_Oldest_TL',
                                                   'Age_Newest_TL',
                                                   'time_since_recent_payment'...
                                                   'CC_enq_L12m', 'PL_enq_L12m',
                                                   'time_since_recent_enq',
                                                   'enq_L3m', ...]),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['MARITALSTATUS', 'GENDER',
                                                   'last_prod_enq2',
                                                   'first_prod_enq2']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['SSC',
                                                                              '12TH',
                                                                              'OTHERS',
                                                                              'GRADUATE',
                                                                              'UNDER '
                                                                              'GRADUATE',
                                                                              'PROFESSIONAL',
                                                                              'POST-GRADUATE']]),
                                                  ['EDUCATION'])])),
                ('svm', SVC())])

In [ ]:
# perform_cv_with_pipe(svm_pipe, X, y, cv=3)

# NOTE:
# 1. Each iteration take ``approx. 50 secs``
# 2. Model score is lesser than RandomForest

In [48]:
print(cls_report_with_pipe(logit_pipe, X_test, y_test))

              precision    recall  f1-score   support

          P1       0.82      0.78      0.80      1224
          P2       0.86      0.96      0.91      6397
          P3       0.65      0.42      0.51      1595
          P4       0.83      0.78      0.80      1309

    accuracy                           0.83     10525
   macro avg       0.79      0.73      0.76     10525
weighted avg       0.82      0.83      0.82     10525



### `RandomForestClassifier`

**Using training data without `Cerdit_Score` column**


In [20]:
# Because we are not using ``Credit_Score`` column;
# We have to update the ColumnTransformer for that
column_trf_wo_cs = ColumnTransformer(
    [
        (
            "scaler",
            StandardScaler(),
            [
                i for i in num_cols if i != "Credit_Score"
            ],  # Removed `Credit_Score` column
        ),
        (
            "ohe",
            OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False),
            OHE_COLS,
        ),
        ("ord", OrdinalEncoder(categories=[EDUCATION_ORD]), ORD_COLS),
    ],
    remainder="passthrough",
)

In [21]:
rf_wo_cs_pipe = Pipeline(
    [
        ("column_trf_wo_cs", column_trf_wo_cs),
        ("model", RandomForestClassifier()),
    ]
)

In [22]:
X_train.columns

Index(['PROSPECTID', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M',
       'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL',
       'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL',
       'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment',
       'num_deliq_6_12mts', 'num_times_60p_dpd', 'num_std_12mts', 'num_sub',
       'num_sub_12mts', 'num_dbt', 'num_dbt_12mts', 'num_lss',
       'recent_level_of_deliq', 'tot_enq', 'CC_enq', 'CC_enq_L12m',
       'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'MARITALSTATUS',
       'EDUCATION', 'AGE', 'GENDER', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr',
       'pct_of_active_TLs_ever', 'pct_opened_TLs_L6m_of_L12m', 'CC_Flag',
       'PL_Flag', 'pct_PL_enq_L6m_of_ever', 'pct_CC_enq_L6m_of_ever',
       'HL_Flag', 'GL_Flag', 'last_prod_enq2', 'first_prod_enq2',
       'Credit_Score'],
      dtype='object')

In [23]:
rf_wo_cs_pipe.fit(X_train.drop(columns=["Credit_Score"]), y_train)

Pipeline(steps=[('column_trf_wo_cs',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  ['pct_tl_closed_L6M',
                                                   'Tot_TL_closed_L12M',
                                                   'pct_tl_open_L12M',
                                                   'pct_tl_closed_L12M',
                                                   'Tot_Missed_Pmnt', 'CC_TL',
                                                   'Gold_TL', 'Home_TL',
                                                   'PL_TL', 'Secured_TL',
                                                   'Unsecured_TL', 'Other_TL',
                                                   'Age_Oldest_TL',
                                                   'Age_Newest_TL',
                                                   'time_since_recent_payme...
                                                   'time_since_recent_enq',
                                                   'enq_L3m', ...]),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['MARITALSTATUS', 'GENDER',
                                                   'last_prod_enq2',
                                                   'first_prod_enq2']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['SSC',
                                                                              '12TH',
                                                                              'OTHERS',
                                                                              'GRADUATE',
                                                                              'UNDER '
                                                                              'GRADUATE',
                                                                              'PROFESSIONAL',
                                                                              'POST-GRADUATE']]),
                                                  ['EDUCATION'])])),
                ('model', RandomForestClassifier())])

In [26]:
perform_cv_with_pipe(rf_wo_cs_pipe, X.drop(columns=["Credit_Score"]), y)

[CV] END ................................ score: (test=0.765) total time=   7.8s
[CV] END ................................ score: (test=0.766) total time=   7.8s
[CV] END ................................ score: (test=0.769) total time=   7.8s
[CV] END ................................ score: (test=0.770) total time=   7.8s
[CV] END ................................ score: (test=0.771) total time=   7.7s
[CV] END ................................ score: (test=0.769) total time=   7.8s
[CV] END ................................ score: (test=0.758) total time=   7.8s
[CV] END ................................ score: (test=0.762) total time=   7.8s


0.7663364685718222

In [28]:
print(cls_report_with_pipe(rf_wo_cs_pipe, X_test, y_test))

              precision    recall  f1-score   support

          P1       0.82      0.70      0.75      1224
          P2       0.79      0.93      0.86      6397
          P3       0.45      0.21      0.28      1595
          P4       0.75      0.70      0.72      1309

    accuracy                           0.77     10525
   macro avg       0.70      0.63      0.65     10525
weighted avg       0.74      0.77      0.74     10525



#### CONCLUSION

In this model I have dropped the `Credit_Score` column the `RandomForestClassifier` model's accuray dropped
significantly.

Hence, we shouldn't use the `Credit_Score` feature for model building because it create a very high bais to model.


# NOTE

1. Try more models `XGBoost`, `LightGBM` and `CatBoost`.
2. ❗ Model training after dropping `Credit_Score` column.
3. 🤯 The models works with encoding the `target_col` with `LabelEncoder`.
